<hr style="border:2px solid #0281c9"> </hr>

<img align="left" alt="ESO Logo" src="http://archive.eso.org/i/esologo.png">  

<div align="center">
  <h1 style="color: #0281c9; font-weight: bold;">ESO Science Archive</h1> 
  <h2 style="color: #0281c9; font-weight: bold;">Jupyter Notebooks</h2>
</div>

<hr style="border:2px solid #0281c9"> </hr>

# **Introduction**

## **ESO Archive Overview**

The ESO Science Archive Facility (SAF) is one of the largest ground-based astronomical data repositories in the world. It contains data from ESO telescopes at La Silla, Paranal, as well as the submillimeter APEX telescope on Llano de Chajnantor and the ALMA array. The archive includes all raw data from La Silla Paranal Observatory, the corresponding calibrations, and a wide range of processed data products, either generated by ESO or contributed by the community. It also hosts selected external datasets, such as raw WFCAM/UKIDSS data from the UKIRT facility in Hawaii. In addition to processed, science-ready data products, the archive provides access to raw observations, which are essential for researchers needing to reprocess data using custom pipelines, verify calibration, or explore previously unpublished results.

## **Raw Data Access: Cross-Instrument vs Instrument-Specific Queries**

ESO provides raw data access through two complementary querying strategies:

- **Cross-instrument queries via `dbo.raw`**  
  This table aggregates metadata across all instruments, enabling broad searches that span multiple facilities. It is well-suited for general use cases, such as retrieving all observations of a target regardless of instrument. Here, however, the metadata is limited to common parameters across instruments like target name, observation date, and exposure time etc. The `dbo.raw` table does not include more instrument-specific parameters.

- **Instrument-specific queries via `ist.<instrument_name>`**  
  Each instrument also has a dedicated table (e.g., `ist.muse`, `ist.midi`, `ist.harps`) offering detailed metadata specific to its configurations and observing modes. These tables expose additional diagnostic parameters (e.g., seeing, binning, readout modes, environmental conditions) not available in `dbo.raw`.

While `dbo.raw` is optimized for uniform queries across the observatory, the `ist.<instrument>` tables are ideal for precision searches tailored to a single instrument’s performance and configuration.

## **Purpose of This Notebook**

This notebook demonstrates how to explore and query **instrument-specific raw data tables** from the ESO Science Archive using **ADQL queries** in python with the ``pyvo`` package.

We will cover how to:

- Discover which instruments have dedicated raw metadata tables  
- Inspect their available columns and field descriptions  
- Construct refined **ADQL queries** to filter based on e.g. instrument-specific parameters such as seeing, airmass, detector binning, and moon illumination

By the end of this notebook, you will be able to identify relevant raw datasets and retrieve observation metadata with full control over instrument-specific filters.

# **Imports**

In [1]:
import numpy as np

from astropy import table
from astropy.table import vstack
from astropy.coordinates import SkyCoord
from astropy.units import Quantity

from pyvo.dal import tap

import matplotlib.pyplot as plt

ESO_TAP_OBS = "http://archive.eso.org/tap_obs"

tapobs = tap.TAPService(ESO_TAP_OBS)

Query for all possible tables in the ESO TAP OBS service

In [ ]:
query = """
        SELECT table_name, description
        FROM TAP_SCHEMA.tables
        ORDER BY table_name

        -- Full explanation:
        -- All TAP services must support a set of (meta) tables, in a schema named TAP_SCHEMA.
        -- These meta-tables describe all tables and columns published through the TAP service.
        -- The meta-tables can be queried themselves via TAP.
        -- Users can discover ESO-published tables or columns by querying the tables in this schema.
        """

result = tapobs.search(query).to_table()
result

table_name,description
object,object
asm.ambient_lasilla,"Table containing the ambient measurements of the ESO seeing monitor (DIMM2) located on a 5m high tower, close to the Schmidt telescope. The seeing is defined as the Full Width Half Maximum of a stellar image observed with a perfect large telescope, at 500nm wavelength and at zenith. It is measured over periods of a few minutes all night long. Time coverage: January 1994 onwards. More info at: https://archive.eso.org/wdb/help/eso/ambient_lasilla.html#ambient"
asm.dimm_paranal,"Table containing the ambient measurements of the Differential Image Motion Monitor (DIMM) channel of the MASS-DIMM instrument. DIMM operates in an extended range of atmospheric conditions which overlap science operation. It provides, along with other measurements, the Paranal reference seeing. Time coverage: 2016-APR-04 12:00:00 UTC onwards. More info at: http://archive.eso.org/wdb/help/eso/ambient_paranal.html#dimm_2016"
asm.historical_ambient_paranal,"Table containing the ambient measurements of the ESO seeing monitor DIMM3 seeing monitor, starting August 1998, and up to the deployment in operations of the new Astronomical Site Monitor (ASM) in April 2016. All seeing values above 2.5 arcsec have been truncated. Time coverage: from 1998-Aug-19 to 2016-Apr-04. More info at: https://archive.eso.org/wdb/help/eso/ambient_paranal.html#dimm_1998"
asm.lhatpros_paranal,"Ambient measurements, since 2014, of the Low Humidity and Temperature Profiling (LHATPRO) microwave radiometers located on Cerro Paranal. Measurements at zenith of: infrared temperature, liquid water path, and precipitable water vapour. Averages over the reported integration time. Three radiometers (identified by their LHATPRO ID: 1, 2, 3) are currently available at different locations named: A, B, C. More info at: http://archive.eso.org/wdb/help/eso/ambient_paranal.html#lhatpro"
asm.lhatpros_paranal_irt,"Ambient measurements, since 2014, of the Low Humidity and Temperature Profiling (LHATPRO) microwave radiometers located on Cerro Paranal. Measurements at provided coordinates of the infrared temperature. Averages over the reported integrtion time (up to 5 seconds, as of 2020). Three radiometers (identified by their LHATPRO ID: 1, 2, 3) are currently available at different locations named: A, B, C. More info at: http://archive.eso.org/wdb/help/eso/ambient_paranal.html#lhatpro"
asm.lhatpros_paranal_profiles,"Ambient measurements, since 2014, of the Low Humidity and Temperature Profiling (LHATPRO) microwave radiometers located on Cerro Paranal. Atmospheric profiles at provided coordinates and at 39 different altitudes from 0m to 10,000m over the VLT platform, of absolute and relative humidity, and infrared temperature. Averages over the reported integration time (typically 60 seconds). More info at: http://archive.eso.org/wdb/help/eso/ambient_paranal.html#lhatpro"
asm.mass_paranal,"Table containing the ambient measurements of the Multi-Aperture Scintillation Sensor located in Paranal. Included measurements like; vertical profile of the turbulence Cn2(h) (6 layers), Isoplanatic angle, Coherence time, etc. More info at: http://archive.eso.org/wdb/help/eso/ambient_paranal.html#mass"
asm.meteo_apex,"Table containing the ambient measurements of the APEX weather station, which is located on a free-standing 6m high tower located about 50m West of the APEX telescope. The sensors are from Vaisala, except the temperature sensor, which is from Stego. The APEX radiometer was built by Radiometer Physics GmbH (MeckenHeim, Germany). Time coverage: January 1st 2006 onwards. More info at: https://archive.eso.org/wdb/help/eso/meteo_apex.html"
asm.meteo_lasilla,"Table containing the ambient measurements of the Vaisala METEOrological station, which was installed at La Silla in February 1985. It includes, on a 30 m high mast, a number of sensors. The following measurements are provided: Wind speed and direction, Temperature, Humidity at different levels. Time co

Now lets only query for the instrument specific `ist` tables 

In [4]:
query = """
        SELECT table_name, description
        FROM TAP_SCHEMA.tables
        WHERE table_name LIKE 'ist.%'
        ORDER BY table_name

        -- Full explanation:
        -- All TAP services must support a set of (meta) tables, in a schema named TAP_SCHEMA.
        -- These meta-tables describe all tables and columns published through the TAP service.
        -- The meta-tables can be queried themselves via TAP.
        -- Users can discover ESO-published tables or columns by querying the tables in this schema.
        """

result = tapobs.search(query).to_table()
result

table_name,description
object,object
ist.alpaca,"Table of the ALPACA raw data. Each record corresponds to one raw file and provides information about the observed target, observing programme, related telescope/instrument setup parameters, and ambient conditions. ALPACA information pages are available at: https://archive.eso.org/docurl?ALPACA All users of the ESO science archive are required to acknowledge the source of the data with an appropriate citation in their publications, as per: https://archive.eso.org/docurl?acknowledgement"
ist.amber,"Table of the AMBER raw data. Each record corresponds to one raw file and provides information about the observed target, observing programme, related telescope/instrument setup parameters, and ambient conditions. AMBER information pages are available at: https://archive.eso.org/docurl?AMBER All users of the ESO science archive are required to acknowledge the source of the data with an appropriate citation in their publications, as per: https://archive.eso.org/docurl?acknowledgement"
ist.apex,"Table of the APEX raw data. Note that the APEX raw data table repeats the same identifier (dp_id) for each of the different receivers used (up to 8); the corresponding records show different rstfrq, skyfrq, and febe values. The APEX information pages are available at: https://archive.eso.org/docurl?APEX All users of the ESO science archive are required to acknowledge the source of the data with an appropriate citation in their publications, as per: https://archive.eso.org/docurl?acknowledgement"
ist.apex_quicklooks,"Table of the APEX quick look data products. Each TAR file contains observing logs and (in most cases) quick reductions of the data, including the scripts used to produce them. APEX information pages are available at: https://archive.eso.org/docurl?APEX All users of the ESO science archive are required to acknowledge the source of the data with an appropriate citation in their publications, as per: https://archive.eso.org/docurl?acknowledgement"
ist.crires,"Table of the CRIRES raw data. Each record corresponds to one raw file and provides information about the observed target, observing programme, related telescope/instrument setup parameters, and ambient conditions. CRIRES information pages are available at: https://archive.eso.org/docurl?CRIRES All users of the ESO science archive are required to acknowledge the source of the data with an appropriate citation in their publications, as per: https://archive.eso.org/docurl?acknowledgement"
ist.efosc,"Table of the EFOSC raw data. Each record corresponds to one raw file and provides information about the observed target, observing programme, related telescope/instrument setup parameters, and ambient conditions. EFOSC information pages are available at: https://archive.eso.org/docurl?EFOSC All users of the ESO science archive are required to acknowledge the source of the data with an appropriate citation in their publications, as per: https://archive.eso.org/docurl?acknowledgement"
ist.eris,"Table of the ERIS raw data. Each record corresponds to one raw file and provides information about the observed target, observing programme, related telescope/instrument setup parameters, and ambient conditions. ERIS information pages are available at: https://archive.eso.org/docurl?ERIS All users of the ESO science archive are required to acknowledge the source of the data with an appropriate citation in their publications, as per: https://archive.eso.org/docurl?acknowledgement"
ist.espresso,"Table of the ESPRESSO raw data. Each record corresponds to one raw file and provides information about the observed target, observing programme, related telescope/instrument setup parameters, and ambient conditions. ESPRESSO information pages are available at: https://archive.eso.org/docurl?ESPRESSO All users of the ESO science archive are required to acknowledge the source of the data with an appropriate citation in their publications, as per: https://archive.eso.

Query for columns in the instrument specific table `ist.muse`

In [8]:
query = """
        SELECT column_name, datatype, arraysize, xtype, unit, description
        from TAP_SCHEMA.columns
        where table_name='ist.muse'
        """

result_istmuse_cols = tapobs.search(query).to_table()
result_istmuse_cols

column_name,datatype,arraysize,xtype,unit,description
object,object,object,object,object,object
access_estsize,long,,,kbyte,"Estimated size of the downloaded file in kilo bytes. It is only ""estimated"" as in general, FITS headers can be patched at download time, making the file size varying with time."
access_url,char,*,,,"A direct URL for downloading the raw data file. If the file is under a proprietary period, access is restricted to authenticated users with the necessary permissions."
ag_fwhmx_avg,float,,,arcsec,Autoguide FWHM mean value in X. Originating from FITS keyword: HIERARCH ESO OCS SGS AG FWHMX AVG.
ag_fwhmy_avg,float,,,arcsec,Autoguide FWHM mean value in Y. Originating from FITS keyword: HIERARCH ESO OCS SGS AG FWHMY AVG.
datalink_url,char,*,,,"A URL that can be used to find the access points of the raw file itself, as well as all its related files. Related files include the calibration files necessary to reduce a science raw frame (see calSelector entries) and, if available, a list of calibrated product(s) generated from it."
date_obs,char,68*,,,"A string (varchar, not a datetime) representing the start time of the observing block (OB). It can be used to match (also using SQL wild cards) the string expressed as ISO 8601 compliant date-time format 'YYYY-MM-DDTHH:MM:SS' (accurate to the second), with a 'T' separating the date and time (e.g., 2000-01-01T00:00:00). Not being a datetime field, date functions like ESO_DATEADD_SEC cannot be used on this column. Originating from FITS keyword: HIERARCH ESO OBS START."
dec,double,,,deg,Declination of the pointing (FK5). Expressed in degrees. Originating from FITS keyword: DEC.
det_binx,int,,,,Binning factor along X. Originating from FITS keyword: HIERARCH ESO DET BINX.
det_biny,int,,,,Binning factor along Y. Originating from FITS keyword: HIERARCH ESO DET BINY.


Now, for comparison, lets query the more generic `dbo.raw` table (which contains all the raw across instruments) to see the difference in available columns compared to the `ist.muse` table.

In [11]:
query = """
        SELECT column_name, datatype, arraysize, xtype, unit, description
        from TAP_SCHEMA.columns
        where table_name='dbo.raw'
        """

result_dboraw_cols = tapobs.search(query).to_table()
result_dboraw_cols

column_name,datatype,arraysize,xtype,unit,description
object,object,object,object,object,object
access_estsize,long,,,kbyte,"Estimated size of the downloaded file in KBytes. It is only ""estimated"" as in general, FITS headers can be patched at download time, making the file size varying with time.)"
access_url,char,*,,,"A URL that can be used to download the raw file; if the file is under proprietary period, only authenticated users with the necessary access rights will be given access to it."
datalink_url,char,*,,,"A URL that can be used to find the access points of the raw file itself, and of all its related files. Related files are: the calibration files necessary to reduce a science raw frame (see calSelector entries), and, if available, a (list of) calibrated product(s) generated out of it.)"
date_obs,char,32*,,,"The content of the DATE_OBS keyword exposed as a string and not a datetime field (ESO_DATEADD_SEC does not work on this column, use exp_start instead)."
dec,double,,,deg,Declination FK5 J2000
dec_pnt,double,,,deg,Pointing Declination FK5 J2000 of the telescope
det_chip1id,char,7*,,,Detector chip identification. Read from keyword DET CHIP1 ID (alias DET CHIP ID). Example: HASCI03
det_chop_ncycles,short,,,,Number of cycles for chopping. Read from keyword DET CHOP NCYCLES. Example: 1
det_dit,float,,,s,Detector Integration Time (Infrared instruments).


In [12]:
# Give me the columns in result_istmuse_cols that are not in result_dboraw_cols
def get_columns_not_in_table(table1, table2):
    """
    Get the columns in table1 that are not in table2
    """
    cols1 = set(table1['column_name'])
    cols2 = set(table2['column_name'])
    return cols1 - cols2

cols_istmuse_not_in_dboraw = get_columns_not_in_table(result_istmuse_cols, result_dboraw_cols)
cols_istmuse_not_in_dboraw

{'ag_fwhmx_avg',
 'ag_fwhmy_avg',
 'det_binx',
 'det_biny',
 'det_read_curname',
 'dimm_fwhm_avg',
 'dimm_fwhm_rms',
 'exptime',
 'ins_opti1_name',
 'ins_opti2_name',
 'lst',
 'moon_illu',
 'night_flag',
 'ob_container_id',
 'ob_container_parent_id',
 'ob_container_parent_type',
 'ob_container_type',
 'ob_start',
 'ob_targ_name',
 'ob_tplno',
 'observing_mode',
 'tel_ambi_temp',
 'tel_ambi_winddir',
 'tel_ambi_windsp',
 'utc'}

Now lets make a query to `ist.muse` that would not be possible with the `dbo.raw` table

In [22]:
query = """
        SELECT *
        FROM ist.muse
        WHERE night_flag = 'NIGHT'
            AND moon_illu < 30
            AND dimm_fwhm_avg < 1.0
            AND exptime > 100
            AND lst between 0 and 6   
        """

result_istmuse = tapobs.search(query).to_table()
result_istmuse

access_estsize,access_url,ag_fwhmx_avg,ag_fwhmy_avg,datalink_url,date_obs,dec,det_binx,det_biny,det_ndit,det_read_curname,dimm_fwhm_avg,dimm_fwhm_rms,dp_cat,dp_id,dp_tech,dp_type,ecl_lat,ecl_lon,exp_start,exposure,exptime,gal_lat,gal_lon,ins_mode,ins_opti1_name,ins_opti2_name,instrument,lambda_max,lambda_min,last_mod_date,lst,mjd_obs,moon_illu,night_flag,ob_container_id,ob_container_parent_id,ob_container_parent_type,ob_container_type,ob_id,ob_name,ob_start,ob_targ_name,ob_tplno,object,observing_mode,origfile,period,pi_coi,prog_id,prog_title,prog_type,ra,release_date,s_region,tel_airm_end,tel_airm_start,tel_alt,tel_ambi_fwhm_end,tel_ambi_fwhm_start,tel_ambi_pres_end,tel_ambi_pres_start,tel_ambi_rhum,tel_ambi_temp,tel_ambi_winddir,tel_ambi_windsp,tel_az,telescope,tpl_expno,tpl_id,tpl_name,tpl_nexp,tpl_start,utc
kbyte,,arcsec,arcsec,,,deg,,,,,arcsec,arcsec,,,,,deg,deg,,s,s,deg,deg,,,,,nm,nm,,s,d,,,,,,,,,,,,,,,,,,,,deg,,,,,deg,arcsec,arcsec,hPa,hPa,%,Celsius,deg,m.s**(-1),deg,,,,,,,s
int64,object,float32,float32,object,object,float64,int32,int32,int32,object,float32,float32,object,object,object,object,float64,float64,object,float32,float32,float64,float64,object,object,object,object,float64,float64,object,float32,float64,int16,object,int32,int32,object,object,int32,object,object,object,int32,object,object,object,int32,object,object,object,object,float64,object,object,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,object,int32,object,object,int32,object,float32
260115,https://dataportal.eso.org/dataPortal/file/MUSE.2022-09-02T05:56:20.456,0.732,0.792,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?MUSE.2022-09-02T05:56:20.456,2022-09-02T05:56:20.455,-5.4429,1,1,1,SCI1.0,0.7074444,0.045997586,SCIENCE,MUSE.2022-09-02T05:56:20.456,IFU,OBJECT,2.201469,199.105136,2022-09-02T05:56:20.457Z,675.0,675.0,-53.135588,63.304004,WFM-AO-N,Blue-Na2,WFM,MUSE,--,--,2022-09-02T06:16:39.227Z,0.524,59824.24745898,-32,night,-999,--,,U,3254647,WFM_W2246_OB10,2022-09-02T05:16:44,W2246-0526,2,W2246-0526,Service,MUSE_WFM-AO_OBS245_0014.fits,109,Assef/ Diaz-Santos/ Blain/ Aravena/ Eisenhardt/ Jun/ Stern/ Elbaz,109.2393.001,Resolving the Gas Kinematics and Environment of the Multiple-Merger Assembly of an Obscured Quasar,Normal,341.53164,2023-09-02T06:16:52Z,POSITION J2000 341.53164000000004 -5.4429,1.132,1.112,63.983,0.69,0.72,742.5,742.6,3.0,9.22,303.0,2.4,134.936,ESO-VLT-U4,3,MUSE_wfm-ao_obs_genericoffsetLGS,Observation in WFM with user defined offsets,4,2022-09-02T05:27:43,21369.0
268407,https://dataportal.eso.org/dataPortal/file/MUSE.2024-11-02T01:54:29.709,0.773,0.715,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?MUSE.2024-11-02T01:54:29.709,2024-11-02T01:54:29.708,-64.05934,1,1,1,SCI1.0,0.573375,0.045134522,SCIENCE,MUSE.2024-11-02T01:54:29.709,IFU,OBJECT,-50.224974,229.705414,2024-11-02T01:54:29.710Z,580.0,580.0,-47.809598,323.006053,WFM-NOAO-N,Blue,WFM,MUSE,--,--,2024-11-02T02:10:16.773Z,1.259,60616.07951052,0,night,3881955,-999,U,G,3883416,WFM_NOAON_IC5244_OB13_D6,2024-11-02T01:49:56,IC5244,2,IC5244,Service,MUSE_WFM-NOAO_OBS307_0007.fits,113,van de Sande / Fraser-McKelvie / Fisher / Hayden / Martig / Bland-Hawthorn / Bolatto / Buck / Buder / Catinella / Ciuca / Cortese / Croom / Falcon-Barroso / Fragkoudi / Freeman / Gerhard / Harborne / Lagos / Leaman / McDermid / Martell / Mendel / Minchev,110.24AS.013,GECKOS: Turning galaxy evolution on its side with deep observations of edge-on galaxies,Large,341.058813,2025-11-02T02:10:36Z,POSITION J2000 341.058813 -64.05934,1.337,1.327,48.869,0.55,0.59,744.7,744.5,10.0,14.13,139.5,3.5,12.253,ESO-VLT-U4,1,MUSE_wfm-noao_obs_genericoffset,Observation in WFM with user defined offsets,6,2024-11-02T01:54:10,6857.0
266037,https://dataportal.eso.org/dataPortal/file/MUSE.2022-09-04T05:48:33.553,0.958,0.929,https://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?MUSE.2022-09-04T05:48:33.553,2022-09-04T05:48:33.552,30.62486,1,1,1,SCI1.0,0.7178,0.

# **Finding Instrument-Specific Columns Matching a FITS Header Keyword**

ESO’s instrument-specific raw data tables (e.g., ist.muse, ist.xshooter) contain many columns that map directly to FITS header keywords. If you’re searching for a specific keyword—like HIERARCH ESO DPR TECH, which appears in many ESO FITS files—you can use the TAP_SCHEMA.columns table to locate which columns store this information.

The following ADQL query searches the descriptions of all columns in all ist.* tables for the string "HIERARCH ESO DPR TECH". This helps you identify the corresponding TAP column name that contains the FITS keyword content.

In [5]:
query = """
        SELECT table_name, column_name, description
        FROM TAP_SCHEMA.columns
        WHERE table_name LIKE 'ist.%'
        AND description LIKE '%HIERARCH ESO DPR TECH%'
        """
result = tapobs.search(query).to_table()
result

table_name,column_name,description
object,object,object
ist.alpaca,dp_tech,Observation technique or methodology employed. Originating from FITS keyword: HIERARCH ESO DPR TECH.
ist.amber,dp_tech,Observation technique or methodology employed. Originating from FITS keyword: HIERARCH ESO DPR TECH.
ist.apex,dp_tech,Observation technique or methodology employed. Originating from FITS keyword: HIERARCH ESO DPR TECH.
ist.crires,dp_tech,Observation technique or methodology employed. Originating from FITS keyword: HIERARCH ESO DPR TECH.
ist.efosc,dp_tech,Observation technique or methodology employed. Originating from FITS keyword: HIERARCH ESO DPR TECH.
ist.eris,dp_tech,Observation technique or methodology employed. Originating from FITS keyword: HIERARCH ESO DPR TECH.
ist.espresso,dp_tech,Observation technique or methodology employed. Originating from FITS keyword: HIERARCH ESO DPR TECH.
ist.fiat,dp_tech,Observation technique or methodology employed. Originating from FITS keyword: HIERARCH ESO DPR TECH.
ist.fors1,dp_tech,Observation technique or methodology employed. Originating from FITS keyword: HIERARCH ESO DPR TECH.
